In [29]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [30]:
exps = glob('Compiled/*')
vcs = glob('Inbal/*/value_counts*')

convert = {
    '590': '10',
    '610': '8',
    '620': '7',
    '630': '6a',
    '640': '6B',
}

In [31]:
vc_dict = {}

for vc in vcs:
    key = vc.split('/')[1]
    test = pd.read_csv(vc)
    vc_dict[key] = test

In [32]:
exp_dict = {}

for exp in exps:

    nm = exp.split('_')[1].split(' ')[0]
    
    with open(exp) as fp:
        #The first 14 lines of the qTable do not contain spectrum data
        for blank in range(0,2):
            fp.readline()
        wave = []
        Q_ext = []
#         Q_abs = []
#         Q_sca = []
        for line in fp:
            ary = line.split(",")
            wave.append(float(ary[0]))
            Q_ext.append(float(ary[1]))


        df = pd.DataFrame({'wave': wave, q: Q_ext})
        exp_dict[nm] = df
        exp_dict[nm] = exp_dict[nm].drop(range(0, 100)).reset_index(drop=True)
#         exp_dict[nm] = exp_dict[nm].iloc[::2, :].reset_index(drop=True)

In [33]:
q = 'Q_sca'
x_dict = {}
for x in glob(f'x_*_new_interp_{q}.csv'):
    test = pd.read_csv(x)
    x_dict[int(x.split('_')[1])] = test

    
yz_dict = {}
for yz in glob(f'yz_*_new_interp_{q}.csv'):
    test = pd.read_csv(yz)
    yz_dict[int(yz.split('_')[1])] = test
    
net_dict = {}
for key in x_dict:
    net_dict[key] = x_dict[key] * (1.0/3.0) + yz_dict[key] * (2.0/3.0)

In [34]:
import numpy as np

best_dict = {}

for peak in convert:
    print(peak)
    exp_temp = exp_dict[peak]
    
    vc_temp = vc_dict[convert[peak]]
    vc_temp = vc_temp.reset_index()
    
    sp = 1.15
    total_count = 0

    test_spectra = exp_dict[peak].copy()
    test_spectra[q] = 0.0
    
    for _, row in vc_temp.iterrows():

        leng = row['Size']
        num = row['Count']

        if leng > 35:
            break

        spdf = net_dict[leng]['%.2f' % sp]
        test_spectra[q] += spdf * num
        total_count += num

    test_spectra[q] = test_spectra[q] / total_count

    plt.plot(test_spectra['wave'], test_spectra[q], label='theory')
    plt.legend()

    plt.title(f'{q}_{sp}nm_spacing')
    plt.xlabel(f'Wavelength (nm)')
    plt.ylabel(f'Intensity (arbitrary units)')
    plt.savefig(f'outimg/{peak}_{q}_sample.png')
    plt.close()
    
    df = pd.DataFrame({'wl': test_spectra['wave'], 'computed': test_spectra[q]})
    df.to_csv(f'outimg/{peak}_{q}_sample.csv')

590
610
620
630
640
